## Задачі

### Завдання 1.0

Використовуючи Python та бібліотеку pandas, порахуйте середній за весь період конверт користувачів з тріального періоду (ті, які заплатили перший платіж) в успішно сплачений другий платіж. Для розрахунку метрики включайте лише користувачів, які мають підписку tenwords_1w_9.99_offer.

### Завдання 1.1

Спрогнозувати LTV за пів року життя користувача використовуючи дані **[файл 1](https://docs.google.com/spreadsheets/d/1J1j1Noq9mIFfXXbivUC35gCEioSIn6rUs8tZe-Rrmvs/edit?gid=1120958046#gid=1120958046).**

LTV потрібно спрогнозувати тільки для підписки tenwords_1w_9.99_offer. 

Рішення має містити:

- розбиття LTV на компоненти
- передбачення компонент на базі історичних даних. (Варто зазначити, що в цьому випадку не вимагається складних математичних моделей чи підходів ML)

Структура даних:

- refunded=True - гроші за транзакцію повернено;

Всі користувачі, що присутні у датасеті виконали перше оформлення підписки за період від початку до кінця датасету. Також датасет містить додаткові колонки country_code (ISO-code країни користувача) та media_source (з якого джерела прийшов користувач), які можуть бути корисними для додаткового завдання 1.2. 

Важливо пояснити, чому ви вибрали саме такий підхід до оцінки LTV і прокоментувати результати. Для процесування даних в цьому завданні можна використовувати Python або Excel.

### Завдання 1.2

Побудувати інформаційну панель (dashboard) використовуючи Tableau/Python та LTV отриманий у завданні 1.1. Потрібно зобразити ROMI поденно для кожного mediasource з можливістю фільтрації по країнах:

Дані з витратами: **[файл 2](https://docs.google.com/spreadsheets/d/1vGyDppIv-IAGeVONsPdetS82MtIYhZa1Rkfig8BM3O8/edit?usp=sharing);**
Cost - витрати на маркетинг у $.

## Імпорт Модулів Python

Спочатку імпортуємо основні модулі, які використовуватимуться в цьому проекті.

Також заглушаємо застереження.

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Завантаження Даних

### Платежі користувачів

Для аналізу наданої інформації завантажуємо набори даних у `DataFrame`. Після завантаження як `DataFrame` дані можна досліджувати та візуалізувати за допомогою Python.

У наступних кроках `User payments.csv` та `Marketing investments.csv` читаються як `DataFrame`, які називаються `df` та `dfm` відповідно. 

Щойно створений `DataFrame` проглядається за допомогою `.head()`, а спеціальна функція `.check()` визначена для оцінки типів даних, унікальних значень і нульових значень для кожного стовпця.

In [2]:
df = pd.read_csv('User payments.csv')

Встановлюємо формат datetime та додаємо колонку місяців

In [3]:
df['purchase_date'] = pd.to_datetime(df['purchase_date'])
df['month'] = df['purchase_date'].dt.month

In [4]:
df.head()

,user_id,product_id,refunded,purchase_date,country_code,media_source,month
0,2589061,tenwords_1w_7.99_7free,False,2020-09-27,JO,Facebook Ads,9
1,2604470,tenwords_1w_9.99_offer,False,2020-09-29,UK,Facebook Ads,9
2,2274467,tenwords_1w_9.99_offer,False,2020-08-18,Organic/Unknown,Organic/Unknown,8
3,2488755,tenwords_lifetime_limited_49.99,False,2020-09-14,PE,Facebook Ads,9
4,2571294,tenwords_1w_9.99_offer,False,2020-09-25,US,Google Ads,9


In [5]:
def check(df):
    list = []
    columns = df.columns
    for col in columns:
        data_types = df[col].dtypes
        total_values = df[col].count()
        unique_values = df[col].nunique()
        null_values = df[col].isnull().sum()
        list.append([col, data_types, total_values, unique_values, null_values])
    df_check = pd.DataFrame(list)
    df_check.columns = ['column', 'data types', 'total values', 'unique values', 'null values']
    return df_check
check(df)

,column,data types,total values,unique values,null values
0,user_id,int64,62874,29321,0
1,product_id,object,62874,3,0
2,refunded,bool,62874,2,0
3,purchase_date,datetime64[ns],62874,92,0
4,country_code,object,62746,200,128
5,media_source,object,62874,4,0
6,month,int32,62874,3,0


### Маркетингові інвестиції

Виконуємо аналогічні функції попереднього `DataFrame`

In [6]:
dfm = pd.read_csv('Marketing investments.csv')

In [7]:
dfm['date'] = pd.to_datetime(dfm['date'])
dfm['month'] = dfm['date'].dt.month

In [8]:
dfm.head(10)

,date,country_code,cost,media_source,month
0,2020-07-01,AD,0.17,Facebook Ads,7
1,2020-07-01,AE,59.30,Facebook Ads,7
2,2020-07-01,AF,1.23,Facebook Ads,7
3,2020-07-01,AG,0.23,Facebook Ads,7
4,2020-07-01,AI,0.18,Facebook Ads,7
5,2020-07-01,AL,1.60,Facebook Ads,7
6,2020-07-01,AM,2.07,Facebook Ads,7
7,2020-07-01,AO,2.63,Facebook Ads,7
8,2020-07-01,AQ,0.01,Facebook Ads,7
9,2020-07-01,AR,6.73,Facebook Ads,7


In [9]:
def check(dfm):
    list = []
    columns = dfm.columns
    for col in columns:
        data_types = dfm[col].dtypes
        total_values = dfm[col].count()
        unique_values = dfm[col].nunique()
        null_values = dfm[col].isnull().sum()
        list.append([col, data_types, total_values, unique_values, null_values])
    df_check = pd.DataFrame(list)
    df_check.columns = ['column', 'data types', 'total values', 'unique values', 'null values']
    return df_check
check(dfm)

,column,data types,total values,unique values,null values
0,date,datetime64[ns],22039,92,0
1,country_code,object,21947,240,92
2,cost,float64,22039,4315,0
3,media_source,object,22039,3,0
4,month,int32,22039,3,0


## Аналіз

### Conversion Rate

Згідно завдання 1.0, створюємо таблицю лише з підписками `tenwords_1w_9.99_offer` і сортуємо від меншого до більшого за *user_id, purchase_date*.

In [10]:
df_2nd_offer = df[df["product_id"] == "tenwords_1w_9.99_offer"]
df_2nd_offer = df_2nd_offer.sort_values(by=['user_id', 'purchase_date'])

Перевіряємо правильність новостворенної таблиці за кількістю значень

In [11]:
count = (df["product_id"] == "tenwords_1w_9.99_offer").sum()
count2 = (df_2nd_offer["product_id"] == "tenwords_1w_9.99_offer").sum()
print(count)
print(count2)

43613
43613


Перевіряємо типи даних, кількість унікальних та нульових значень

In [12]:
check(df_2nd_offer)

,column,data types,total values,unique values,null values
0,user_id,int64,43613,18014,0
1,product_id,object,43613,1,0
2,refunded,bool,43613,2,0
3,purchase_date,datetime64[ns],43613,92,0
4,country_code,object,43517,186,96
5,media_source,object,43613,4,0
6,month,int32,43613,3,0


Додаємо дату першого платежу, де:
- groupby("user_id") групує дані по користувачах.
- transform("min") застосовує функцію мінімуму до кожної групи, повертаючи серію, де для кожного користувача буде зазначено його перше значення (мінімальна дата покупки).

In [13]:
df_2nd_offer["first_payment_date"] = df_2nd_offer.groupby("user_id")["purchase_date"].transform("min")
df_2nd_offer.head()

,user_id,product_id,refunded,purchase_date,country_code,media_source,month,first_payment_date
6970,27637,tenwords_1w_9.99_offer,False,2020-07-05,PL,Facebook Ads,7,2020-07-05
38650,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,8,2020-08-25
61235,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,8,2020-08-25
47316,31921,tenwords_1w_9.99_offer,False,2020-08-25,FR,Facebook Ads,8,2020-08-25
31438,34244,tenwords_1w_9.99_offer,False,2020-09-07,NL,Facebook Ads,9,2020-09-07


Додаємо для кожного користувача (перебір по колонці user_id в df_2nd_offer):
- Якщо поточний user_id дорівнює попередньому (тобто це той самий користувач, що й раніше), то додається значення 9.99 до списку cost.
- Якщо user_id змінюється (це новий користувач), додається значення 0.50.

Таким чином, кожному користувачу присвоюється 0.50 USD, якщо це перша оплата і 9.99 USD за наступні.

In [14]:
cost = []
previous_user_id = 0

for user_id in df_2nd_offer["user_id"]:
    if user_id == previous_user_id:
        cost.append(9.99)
    else:
        cost.append(0.50)
    previous_user_id = user_id

df_2nd_offer["revenue, USD"] = cost

df_2nd_offer.head()

,user_id,product_id,refunded,purchase_date,country_code,media_source,month,first_payment_date,"revenue, USD"
6970,27637,tenwords_1w_9.99_offer,False,2020-07-05,PL,Facebook Ads,7,2020-07-05,0.50
38650,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,8,2020-08-25,0.50
61235,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,8,2020-08-25,9.99
47316,31921,tenwords_1w_9.99_offer,False,2020-08-25,FR,Facebook Ads,8,2020-08-25,0.50
31438,34244,tenwords_1w_9.99_offer,False,2020-09-07,NL,Facebook Ads,9,2020-09-07,0.50


Функція `second_payment_check` перевіряє, чи є у користувача більше ніж один платіж. Таким чином, у колонці `second_payment` для кожного користувача позначається, чи є у нього другий платіж (True, якщо більше одного платежу, і False, якщо лише один).

In [15]:
def second_payment_check(group):
    if len(group) > 1:
        return [True] * len(group)
    return [False] * len(group)

grouped_results = df_2nd_offer.groupby("user_id").apply(second_payment_check)
df_2nd_offer["second_payment"] = grouped_results.explode().values

df_2nd_offer.head()

,user_id,product_id,refunded,purchase_date,country_code,media_source,month,first_payment_date,"revenue, USD",second_payment
6970,27637,tenwords_1w_9.99_offer,False,2020-07-05,PL,Facebook Ads,7,2020-07-05,0.50,False
38650,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,8,2020-08-25,0.50,True
61235,31549,tenwords_1w_9.99_offer,False,2020-08-25,CO,Facebook Ads,8,2020-08-25,9.99,True
47316,31921,tenwords_1w_9.99_offer,False,2020-08-25,FR,Facebook Ads,8,2020-08-25,0.50,False
31438,34244,tenwords_1w_9.99_offer,False,2020-09-07,NL,Facebook Ads,9,2020-09-07,0.50,False


**Conversion Rate = Second Payment Users / First Payment Users**

- Кількість користувачів з першим платежем, це всі унікальні значення user_id з product_id, що дорівнює tenwords_1w_9.99_offer.
- Кількість користувачів з другим платежем, це всі унікальні значення user_id, у яких більше одного платежу.
- Згідно розрахунку **Коефіціент конверсії** складає `42.12%`

In [16]:
first_payment_users = df_2nd_offer["user_id"].nunique()
second_payment_users = df_2nd_offer[df_2nd_offer["second_payment"] == True]["user_id"].nunique()
conversion_rate = second_payment_users / first_payment_users

print(f"Кількість користувачів з першим платежем: {first_payment_users}")
print(f"Кількість користувачів з другим платежем: {second_payment_users}")
print(f"Середній конверт у другий платіж: {conversion_rate:.2%}")

Кількість користувачів з першим платежем: 18014
Кількість користувачів з другим платежем: 7587
Середній конверт у другий платіж: 42.12%


### LTV (Lifetime Value)

**Lifetime Value = (Average Revenue Per User * Time)**

- ARPU (Average Revenue Per User) - середній дохід на одного клієнта
- Time - тривалість життєвого циклу клієнта

Фільтруємо на прибуток без повернень

In [17]:
df_valid_payments = df_2nd_offer[df_2nd_offer["refunded"] == False]

Рахуємо Загальний дохід, Кількість унікальних користувачів та Обчислюємо ARPU. 

З розрахунку зваженого середнього маємо `ARPU: 14.39 USD`

In [18]:
total_revenue = df_valid_payments["revenue, USD"].sum()
unique_users = df_valid_payments["user_id"].nunique()
ARPU = total_revenue / unique_users

print(f"total_revenue: {total_revenue:.2f} USD")
print(f"unique_users: {unique_users}")
print(f"ARPU: {ARPU:.2f} USD")

total_revenue: 251924.06 USD
unique_users: 17510
ARPU: 14.39 USD


In [19]:
df_2nd_offer_ltv = df_valid_payments.pivot_table(
    index="month",
    values=["user_id", "revenue, USD"],
    aggfunc={
        "user_id": pd.Series.nunique,
        "revenue, USD": "sum",
    },
)

df_2nd_offer_ltv["revenue_per_user"] = (
    df_2nd_offer_ltv["revenue, USD"] / df_2nd_offer_ltv["user_id"]
)

df_2nd_offer_ltv.loc["Total"] = df_2nd_offer_ltv.sum(numeric_only=True)
df_2nd_offer_ltv.loc["Total", "user_id"] = df_valid_payments["user_id"].nunique()

df_2nd_offer_ltv.loc["Total", "revenue_per_user"] = (
    df_2nd_offer_ltv.loc["Total", "revenue, USD"]
    / df_2nd_offer_ltv.loc["Total", "user_id"]
)

df_2nd_offer_ltv

,"revenue, USD",user_id,revenue_per_user
month,,,
7,37534.54,6225.0,6.029645
8,93260.47,7327.0,12.728329
9,121129.05,7994.0,15.152496
Total,251924.06,17510.0,14.387439


ARPU множимо на пів року життя користувача і знаходимо `LTV: 86.32 USD`

In [20]:
LTV = ARPU * 6

print(f"Прогнозований LTV за 6 місяців: {LTV:.2f} USD")

Прогнозований LTV за 6 місяців: 86.32 USD


### ROMI (Return on Marketing Investment)

**ROMI = ((Total Revenue - Total Marketing Cost) / Total Marketing Cost) * 100**

In [21]:
dfm.head()

,date,country_code,cost,media_source,month
0,2020-07-01,AD,0.17,Facebook Ads,7
1,2020-07-01,AE,59.30,Facebook Ads,7
2,2020-07-01,AF,1.23,Facebook Ads,7
3,2020-07-01,AG,0.23,Facebook Ads,7
4,2020-07-01,AI,0.18,Facebook Ads,7


Відображаємо загальний дохід, який попередньо був розрахований. 

Також розраховуємо Загальні маркетингові витрати і `ROMI`, який становить `-48.38%`

In [22]:
total_marketing_cost = dfm["cost"].sum()
ROMI = ((total_revenue - total_marketing_cost)/total_marketing_cost)*100

print(f"Загальний дохід: {total_revenue:.2f} USD")
print(f"Загальні маркетингові витрати: {total_marketing_cost:.2f} USD")
print(f"ROMI: {ROMI:.2f}%")

Загальний дохід: 251924.06 USD
Загальні маркетингові витрати: 488015.90 USD
ROMI: -48.38%


In [23]:
dfm_expbysource = dfm.pivot_table(index='media_source', columns='month', values='cost', aggfunc='sum')
dfm_expbysource

month,7,8,9
media_source,,,
ByteDance,162.24,9181.14,15533.90
Facebook Ads,158430.68,149170.79,108283.62
Google Ads,1190.19,9026.82,37036.52


In [24]:
dfm_expbymonth = dfm.pivot_table(index='month', values='cost', aggfunc='sum')
dfm_expbymonth

,cost
month,
7,159783.11
8,167378.75
9,160854.04


In [25]:
dfm_expbysource1 = dfm.pivot_table(index='media_source', values='cost', aggfunc='sum')
dfm_expbysource1

,cost
media_source,
ByteDance,24877.28
Facebook Ads,415885.09
Google Ads,47253.53


## Висновок

У цьому дослідженні було проведено аналіз конверсії користувачів з тріального періоду в платну підписку, спрогнозовано значення LTV (Lifetime Value) на основі історичних даних та розраховано значення ROMI. 

Основна увага була зосереджена на підписці tenwords_1w_9.99_offer, що дозволило зробити точніші розрахунки ключових показників.

- Завдання 1.0. Розрахунок середнього за весь період конверту користувачів з тріального періоду в сплачений другий платіж.
    - **Коефіціент конверсії** складає `42.12%`.
- Завдання 1.1. Спрогнозувати LTV за пів року життя користувача.
    - **Середній дохід на користувача** складає `14.39 USD`.
    - **Піврічний Lifetime Value** складає `86.32 USD`.
- Завдання 1.2. Зобразити ROMI
    - **Рентабельність інвестицій у маркетинг** складає `-48.38%`
    - Додано візуалізацію в **[Tableau](https://public.tableau.com/app/profile/illia.ukraintsev/viz/ROMIbydays/ROMI?publish=yes).**